In [1]:
#####Robust Network Co-Clustering#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import gensim
import itertools
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns
import time

#np.random.seed(20)

C:\Users\sana\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no):
    z_id = np.argmax((np.cumsum(pr, axis=1) > np.random.rand(n).reshape(n, 1)), axis=1)
    Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
    return Z

In [3]:
####データの発生####
##データの設定
k1 = 15   #ユーザーのクラスタ数
k21 = 12   #アイテムの関係オブジェクトのクラスタ数
k22 = 8   #アイテムの無関係オブジェクトのクラスタ数
hh = 10000   #ユーザー数
item = 5000   #アイテム数
pt = np.random.poisson(np.random.gamma(27.5, 1/0.2, hh))   #ユーザーあたりのレコード数
f = np.sum(pt)   #総レコード数

In [4]:
##IDとインデックスの設定
#IDの設定
user_id = np.repeat(range(hh), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(hh)])))

#インデックスの設定
index = np.array(range(f))
user_index = [i for i in range(hh)]
user_vec = [i for i in range(hh)]
for i in range(hh):
    user_index[i] = index[user_id==i]
    user_vec[i] = np.repeat(1, len(user_index[i]))

In [5]:
##アイテムの割当を生成
#セグメント割当を生成
topic = 25
phi = np.random.dirichlet(np.repeat(0.5, item), topic)
theta = np.random.dirichlet(np.repeat(2.5, topic), hh)
z = np.dot(np.array([np.random.multinomial(1, theta[i, :], 1) for i in range(hh)]).reshape(hh, topic), range(topic))

#多項分布からアイテムを生成
item_id = np.zeros(f, dtype='int')
for i in range(hh):
    if i%1000==0:
        print(i)
    item_id[user_index[i]] = np.dot(np.random.multinomial(1, phi[z[i], :], pt[i]), range(item))
    
#インデックスの設定
item_index = [j for j in range(item)]
item_vec = [j for j in range(item)]
for j in range(item):
    item_index[j] = index[item_id==j]
    item_vec[j] = np.repeat(1, len(item_index[j]))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [6]:
##セグメント割当を生成
#関係有無の潜在変数rを生成
tau = 0.4; taut = tau
r = np.random.binomial(1, tau, item); rt = r
index_r0 = np.array(range(item))[r==0]; n0 = index_r0.shape[0]
index_r1 = np.array(range(item))[r==1]; n1 = index_r1.shape[0]

#ユーザーセグメントを生成
alpha1 = np.random.dirichlet(np.repeat(5.0, k1), 1).reshape(-1); alphat1 = alpha1
Z1 = np.random.multinomial(1, alpha1, hh)
z_vec1 = np.dot(Z1, np.arange(k1))

#アイテムセグメントを生成
alpha21 = np.random.dirichlet(np.repeat(5.0, k21), 1).reshape(-1); alphat21 = alpha21
alpha22 = np.random.dirichlet(np.repeat(5.0, k22), 1).reshape(-1); alphat22 = alpha22
Z21 = np.zeros((hh, k21)); Z22 = np.zeros((hh, k22))
Z21[index_r1, ] = np.random.multinomial(1, alpha21, n1)
Z22[index_r0, ] = np.random.multinomial(1, alpha22, n0)
z_vec21 = np.array(np.dot(Z21, np.arange(k21)), dtype="int")
z_vec22 = np.array(np.dot(Z22, np.arange(k22)), dtype="int")

##ベータ分布からセグメントごとの関係確率を生成
#関係オブジェクトのパラメータを生成
alpha1 = 0.2; beta1 = 0.5
theta = np.random.beta(alpha1, beta1, k1*k21).reshape(k1, k21)
thetat = theta

#無関係オブジェクトのパラメータを生成
alpha2 = 0.6; beta2 = 1.0
phi = np.random.beta(alpha2, beta2, k22)
phit = phi

##関係データを生成
#セグメント割当から関係確率を設定
r_vec = r[item_id]
Prob = r_vec * np.sum(theta[z_vec1[user_id], ] * Z21[item_id, ], axis=1) + (1-r_vec) * phi[z_vec22[item_id]]

#ベルヌーイ分布から関係データを生成
y = np.random.binomial(1, Prob, f)
y_vec = y.reshape(f, 1)
index_y0 = np.array(range(f))[y==0]
index_y1 = np.array(range(f))[y==1]
n0 = np.sum(1-y)
n1 = np.sum(y)

In [7]:
####ギブスサンプリングでRobust Network Co-Clusteringをサンプリング#####
##アルゴリズムの設定
LL1 = -100000000   #対数尤度の初期値
R = 2000
keep = 2  
iter = 0
burnin = 500/keep
disp = 10

In [8]:
##事前分布の設定
#混合率の事前分布
s0 = 0.1; v0 = 0.2
alpha01 = 0.25
alpha02 = 0.25

#モデルパラメータの事前分布
er = 0.00001
beta11 = 0.2; beta12 = 0.2
beta21 = 0.2; beta22 = 0.2

In [9]:
##パラメータの真値
#混合率の真値
tau = taut
alpha1 = alphat1
alpha21 = alphat21
alpha22 = alphat22

#モデルパラメータの真値
theta = thetat
phi = phit

#潜在変数の真値
r = rt 
Zi1 = Z1
Zi21 = Z21
Zi22 = Z22
z_vec1 = np.dot(Zi1, np.arange(k1))
z_vec21 = np.array(np.dot(Zi21, np.arange(k21)), dtype="int")
z_vec22 = np.array(np.dot(Zi22, np.arange(k22)), dtype="int")

In [10]:
##パラメータの初期値
#混合率の初期値
tau = 0.4
alpha1 = np.random.dirichlet(np.repeat(10.0, k1), 1).reshape(-1)
alpha21 = np.random.dirichlet(np.repeat(10.0, k21), 1).reshape(-1)
alpha22 = np.random.dirichlet(np.repeat(10.0, k22), 1).reshape(-1)

#モデルパラメータの初期値
theta = np.random.beta(10.0, 20.0, k1*k21).reshape(k1, k21)
phi = np.random.beta(10.0, 20.0, k22)

#潜在変数の初期値
r = np.random.binomial(1, tau, item)
Zi1 = np.random.multinomial(1, alpha1, hh)
Zi21 = np.random.multinomial(1, alpha21, item) * r.reshape(item, 1)
Zi22 = np.random.multinomial(1, alpha22, item) * (1-r.reshape(item, 1))
z_vec1 = np.dot(Zi1, np.arange(k1))
z_vec21 = np.array(np.dot(Zi21, np.arange(k21)), dtype="int")
z_vec22 = np.array(np.dot(Zi22, np.arange(k22)), dtype="int")

In [11]:
##パラメータの格納用配列
THETA = np.zeros((k1, k21, int(R/keep)))
PHI = np.zeros((int(R/keep), k22))
TAU = np.zeros(int(R/keep))
ALPHA1 = np.zeros((int(R/keep), k1))
ALPHA21 = np.zeros((int(R/keep), k21))
ALPHA22 = np.zeros((int(R/keep), k22))
R_SEG = np.zeros((f))
SEG1 = np.zeros((hh, k1, int(R/keep)))
SEG21 = np.zeros((item, k21, int(R/keep)))
SEG22 = np.zeros((item, k22, int(R/keep)))

In [12]:
##対数尤度の基準値
#ユニグラムモデルの対数尤度
LLst = np.sum(y*np.log(np.mean(y)) + (1-y)*np.log(np.mean(1-y)))
print(LLst)

#真値での対数尤度
r_vec = rt[item_id]
LLbest11 = np.sum(y[r_vec==1] * np.sum(np.log(thetat)[np.dot(Z1, range(k1))[user_id], ] * Z21[item_id, ], axis=1)[r_vec==1])
LLbest12 = np.sum((1-y[r_vec==1]) * np.sum(np.log(1-thetat)[np.dot(Z1, range(k1))[user_id], ] * Z21[item_id, ], axis=1)[r_vec==1])
LLbest21 = np.sum(y[r_vec==0] * np.log(phit)[np.array(np.dot(Z22, range(k22)), dtype="int")[item_id[r_vec==0]]])
LLbest22 = np.sum((1-y[r_vec==0]) * np.log(1-phit)[np.array(np.dot(Z22, range(k22)), dtype="int")[item_id[r_vec==0]]])
LLbest = LLbest11 + LLbest12 + LLbest21 + LLbest22
print(LLbest)

-924686.5519537947
-520789.58032304246


In [13]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    
    ##潜在変数rをサンプリング
    #無関係オブジェクトのクラスタごとの対数尤度
    phi_dt = np.zeros((f, k22))
    phi_dt[index_y1, ] = np.log(np.repeat(phi, n1).reshape(n1, k22, order="F"))
    phi_dt[index_y0, ] = np.log(np.repeat(1-phi, n0).reshape(n0, k22, order="F"))

    #関係オブジェクトのクラスタごとの対数尤度
    theta_dt = np.zeros((f, k21))
    theta_dt[index_y1, ] = np.log(theta)[z_vec1[user_id[index_y1]], ]
    theta_dt[index_y0, ] = np.log(1-theta)[z_vec1[user_id[index_y0]], ]

    #クラスタごとの対数尤度の和
    LLi0 = np.zeros((item, k22)); LLi1 = np.zeros((item, k21))
    for j in range(item):
        LLi0[j] = np.dot(item_vec[j], phi_dt[item_index[j], ])
        LLi1[j] = np.dot(item_vec[j], theta_dt[item_index[j], ])

    #潜在変数rの割当確率
    LLi_max = np.max(np.hstack((LLi0, LLi1)), axis=1).reshape(item, 1)
    Li0 = np.sum(np.repeat(alpha22, item).reshape(item, k22) * np.exp(LLi0 - LLi_max), axis=1)
    Li1 = np.sum(np.repeat(alpha21, item).reshape(item, k21) * np.exp(LLi1 - LLi_max), axis=1)
    Prob = np.hstack(((tau*Li1).reshape(item, 1), ((1-tau)*Li0).reshape(item, 1))) / (tau*Li1 + (1-tau)*Li0).reshape(item, 1)
    Prob = ((Prob + er) / np.sum(Prob + er, axis=1).reshape(item, 1))[:, 0]

    #ベルヌーイ分布からrをサンプリング
    r = np.random.binomial(1, Prob, item)
    index_r1 = np.arange(item)[r==1]; rn1 = index_r1.shape[0]
    index_r0 = np.delete(np.arange(item), index_r1); rn0 = index_r0.shape[0]
    
    #ベータ分布から混合率を更新
    tau = np.random.beta(np.sum(r) + s0, item - np.sum(r) + v0, 1)
    

    ##アイテムのクラスタをサンプリング
    #無関係オブジェクトのクラスタをサンプリング
    Zi22 = np.zeros((item, k22))
    LLi_max = np.max(LLi0[index_r0, ], axis=1).reshape(rn0, 1)
    Li = np.repeat(alpha22, rn0).reshape(rn0, k22, order="F") * np.exp(LLi0[index_r0, ] - LLi_max)
    Prob = Li / np.sum(Li, axis=1).reshape(rn0, 1); Prob = (Prob + er) / np.sum(Prob + er, axis=1).reshape(rn0, 1)
    Zi22[index_r0, ] = np.array(rmnom(Prob, rn0, k22, np.arange(rn0)).todense())
    z_vec22 = np.array(np.dot(Zi22, np.arange(k22)), dtype="int")

    #関係オブジェクトのクラスタをサンプリング
    Zi21 = np.zeros((item, k21))
    LLi_max = np.max(LLi1[index_r1, ], axis=1).reshape(rn1, 1)
    Li = np.repeat(alpha21, rn1).reshape(rn1, k21, order="F") * np.exp(LLi1[index_r1, ] - LLi_max)
    Prob = Li / np.sum(Li, axis=1).reshape(rn1, 1); Prob = (Prob + er) / np.sum(Prob + er, axis=1).reshape(rn1, 1)
    Zi21[index_r1, ] = np.array(rmnom(Prob, rn1, k21, np.arange(rn1)).todense())
    z_vec21 = np.array(np.dot(Zi21, np.arange(k21)), dtype="int")

    ##ユーザーのクラスタをサンプリング
    #アイテムのクラスタの条件付き尤度を設定
    theta_T = theta.T
    LLi = np.zeros((hh, k1)); Lho = np.zeros((f, k1))
    Lho[index_y1, ] = np.log(theta_T[z_vec21[item_id[index_y1]], ]) *  r[item_id[index_y1]].reshape(n1, 1) 
    Lho[index_y0, ] = np.log(1 - theta_T[z_vec21[item_id[index_y0]], ]) * r[item_id[index_y0]].reshape(n0, 1)
    for i in range(hh):
        LLi[i, ] = np.dot(user_vec[i], Lho[user_index[i], ])

    #ベルヌーイ分布からクラスタをサンプリング
    Li = np.repeat(alpha1, hh).reshape(hh, k1, order="F") * np.exp(LLi - np.max(LLi, axis=1).reshape(hh, 1))
    Prob = Li / np.sum(Li, axis=1).reshape(hh, 1); Prob = (Prob + er) / np.sum(Prob + er, axis=1).reshape(hh, 1)
    Zi1 = np.array(rmnom(Prob, hh, k1, np.arange(hh)).todense())
    z_vec1 = np.array(np.dot(Zi1, np.arange(k1)), dtype="int")


    ##ベータ分布から関係オブジェクトのパラメータをサンプリング
    #データの設定
    r_vec = r[item_id]
    y_relation = y * r_vec
    Zi_dt1 = Zi1[user_id, ]
    Zi_dt2 = Zi21[item_id, ]

    for j in range(k21):
        #セグメント割当の頻度を数える
        s = np.dot(y_relation * Zi_dt1[:, j], Zi_dt2)
        n = np.dot(r_vec * Zi_dt1[:, j], Zi_dt2)

        #ベータ分布からパラメータをサンプリング
        tau1 = s + beta11
        tau2 = n - s + beta12
        theta[j, ] = np.random.beta(tau1, tau2, k21)

    ##ベータ分布から無関係オブジェクトのパラメータをサンプリング
    #データの設定
    Zi22_dt = Zi22[item_id, ]

    #セグメント割当の頻度を数える
    s = np.sum(y_vec * Zi22_dt, axis=0)
    n = np.sum(Zi22_dt, axis=0)

    #ベータ分布からパラメータをサンプリング
    tau1 = s + beta21
    tau2 = n - s + beta22
    phi = np.random.beta(tau1, tau2, k22)


    ##混合率をサンプリング
    #関係オブジェクトの混合率
    alpha1 = np.random.dirichlet(np.sum(Zi1, axis=0) + alpha01, 1).reshape(-1)
    alpha21 =  np.random.dirichlet(np.sum(Zi21, axis=0) + alpha01, 1).reshape(-1)

    #無関係オブジェクトの混合率
    alpha22 =  np.random.dirichlet(np.sum(Zi22, axis=0) + alpha02, 1).reshape(-1)


    ##パラメータの格納とサンプリング結果の表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = rp//keep
        THETA[:, :, mkeep] = theta
        PHI[mkeep, ] = phi
        TAU[mkeep] = tau
        ALPHA1[mkeep, ] = alpha1
        ALPHA21[mkeep, ] = alpha21
        ALPHA22[mkeep, ] = alpha22
        SEG1[:, :, mkeep] = Zi1
        SEG21[:, :, mkeep] = Zi21
        SEG22[:, :, mkeep] = Zi22        
        
    #トピック割当はバーンイン期間を超えたら格納
    if rp%keep==0 & rp >= burnin:
        R_SEG = R_SEG + r
        
    if rp%disp==0:
        #対数尤度の和を計算
        r_vec = r[item_id]
        LL11 = np.sum(y[r_vec==1] * np.sum(np.log(theta)[np.dot(Zi1, range(k1))[user_id], ] * Zi21[item_id, ], axis=1)[r_vec==1])
        LL12 = np.sum((1-y[r_vec==1]) * np.sum(np.log(1-theta)[np.dot(Zi1, range(k1))[user_id], ] * Zi21[item_id, ], axis=1)[r_vec==1])
        LL21 = np.sum(y[r_vec==0] * np.log(phi)[np.array(np.dot(Zi22, range(k22)), dtype="int")[item_id[r_vec==0]]])
        LL22 = np.sum((1-y[r_vec==0]) * np.log(1-phi)[np.array(np.dot(Zi22, range(k22)), dtype="int")[item_id[r_vec==0]]])
        LL = LL11 + LL12 + LL21 + LL22

        #サンプリング結果の表示
        print(rp)
        print(np.round(np.vstack((phi, phit)), 3))
        print(np.round(np.array([LL, LLst, LLbest]), 3))

0
[[0.086 0.19  0.249 0.666 0.514 0.446 0.345 0.309]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-792822.462 -924686.552 -520789.58 ]
10
[[0.035 0.194 0.207 0.91  0.479 0.432 0.375 0.353]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542276.978 -924686.552 -520789.58 ]
20
[[0.035 0.194 0.211 0.91  0.49  0.429 0.37  0.357]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542519.25  -924686.552 -520789.58 ]
30
[[0.031 0.196 0.203 0.911 0.075 0.434 0.376 0.357]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542099.713 -924686.552 -520789.58 ]
40
[[0.022 0.198 0.208 0.91  0.057 0.433 0.38  0.346]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-541805.267 -924686.552 -520789.58 ]
50
[[0.023 0.195 0.206 0.911 0.056 0.43  0.376 0.352]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-541593.553 -924686.552 -520789.58 ]
60
[[0.022 0.191 0.207 0.909 0.054 0.43  0.373 0.36 ]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-541749.62  -924686.552 -52078

C:\Users\sana\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log
  del sys.path[0]
C:\Users\sana\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: RuntimeWarning: divide by zero encountered in log
C:\Users\sana\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in multiply
C:\Users\sana\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until


80
[[0.022 0.178 0.21  0.91  0.055 0.433 0.362 0.331]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-712379.647 -924686.552 -520789.58 ]
90
[[0.021 0.19  0.21  0.911 0.058 0.431 0.372 0.371]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-555308.613 -924686.552 -520789.58 ]
100
[[0.022 0.198 0.204 0.911 0.053 0.425 0.364 0.378]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-554804.31  -924686.552 -520789.58 ]
110
[[0.022 0.194 0.216 0.911 0.054 0.427 0.361 0.392]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-550110.406 -924686.552 -520789.58 ]
120
[[0.023 0.194 0.207 0.911 0.055 0.428 0.363 0.395]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-547966.587 -924686.552 -520789.58 ]
130
[[0.022 0.205 0.195 0.91  0.055 0.429 0.373 0.399]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-548010.763 -924686.552 -520789.58 ]
140
[[0.023 0.201 0.199 0.909 0.057 0.429 0.368 0.418]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-547903.906 -924686.552 

650
[[0.022 0.196 0.21  0.936 0.055 0.432 0.373 0.884]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542403.888 -924686.552 -520789.58 ]
660
[[0.023 0.195 0.208 0.934 0.055 0.435 0.371 0.877]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-541942.851 -924686.552 -520789.58 ]
670
[[0.022 0.191 0.207 0.936 0.056 0.44  0.373 0.882]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542227.675 -924686.552 -520789.58 ]
680
[[0.022 0.191 0.213 0.935 0.056 0.44  0.372 0.884]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542379.679 -924686.552 -520789.58 ]
690
[[0.023 0.19  0.207 0.934 0.057 0.437 0.37  0.881]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-541957.001 -924686.552 -520789.58 ]
700
[[0.022 0.18  0.206 0.937 0.056 0.439 0.37  0.883]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-541856.868 -924686.552 -520789.58 ]
710
[[0.023 0.185 0.207 0.935 0.057 0.434 0.365 0.881]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542003.987 -924686.55

1220
[[0.022 0.166 0.2   0.931 0.055 0.435 0.368 0.875]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542071.085 -924686.552 -520789.58 ]
1230
[[0.023 0.149 0.202 0.935 0.059 0.438 0.371 0.882]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542466.04  -924686.552 -520789.58 ]
1240
[[0.023 0.144 0.202 0.935 0.055 0.437 0.368 0.881]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542017.765 -924686.552 -520789.58 ]
1250
[[0.023 0.146 0.203 0.935 0.057 0.438 0.37  0.88 ]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542149.904 -924686.552 -520789.58 ]
1260
[[0.023 0.132 0.201 0.933 0.054 0.435 0.372 0.88 ]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542148.339 -924686.552 -520789.58 ]
1270
[[0.021 0.2   0.202 0.937 0.054 0.434 0.369 0.879]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542366.985 -924686.552 -520789.58 ]
1280
[[0.022 0.149 0.2   0.936 0.057 0.432 0.369 0.881]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542162.296 -92

1790
[[0.022 0.125 0.202 0.936 0.054 0.434 0.369 0.881]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542165.998 -924686.552 -520789.58 ]
1800
[[0.023 0.135 0.203 0.936 0.057 0.433 0.369 0.878]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542005.205 -924686.552 -520789.58 ]
1810
[[0.022 0.142 0.202 0.935 0.056 0.437 0.368 0.879]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542212.016 -924686.552 -520789.58 ]
1820
[[0.023 0.11  0.202 0.936 0.057 0.438 0.371 0.877]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542087.728 -924686.552 -520789.58 ]
1830
[[0.023 0.122 0.201 0.936 0.056 0.434 0.367 0.879]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542795.829 -924686.552 -520789.58 ]
1840
[[0.023 0.157 0.2   0.934 0.056 0.439 0.371 0.878]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542559.75  -924686.552 -520789.58 ]
1850
[[0.022 0.154 0.203 0.935 0.056 0.436 0.368 0.879]
 [0.368 0.421 0.057 0.439 0.022 0.202 0.931 0.883]]
[-542080.512 -92

1380531